In [14]:
from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.entities import Environment, BuildContext

from azure.identity import ClientSecretCredential, DefaultAzureCredential

from pathlib import Path

# read .env file as environment variables
with open("/home/azureuser/cloudfiles/code/Users/alibina/Mlops-cli2-classic/data-science/experiment/.env", "r") as f:
    for line in f:
        if line.strip() != "":
            key, value = line.strip().split("=", 1)
            os.environ[key] = value

# instantiate the MLClient using service principal authentication
credential = ClientSecretCredential(client_id=os.environ["AZURE_CLIENT_ID"],
                                    client_secret=os.environ["AZURE_CLIENT_SECRET"],
                                    tenant_id=os.environ["AZURE_TENANT_ID"])

ml_client = MLClient.from_config(credential=credential)
print(ml_client)


Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/alibina8/code/Users/alibina/config.json


MLClient(credential=<azure.identity._credentials.client_secret.ClientSecretCredential object at 0x7fc8b8176ef0>,
         subscription_id=f804f2da-c27b-45ac-bf80-16d4d331776d,
         resource_group_name=rg-mlopsv2clas-505prod,
         workspace_name=mlw-mlopsv2clas-505prod)


In [15]:
# creat a aml cluster
cpu_cluster_name = "cpu-cluster"
try:
    cpu_cluster = ml_client.compute.get(cpu_cluster_name)
    print("Found existing cluster, use it.")
except:
    print("Creating a new cluster...")
    compute_config = ml_client.compute.configuration.CpuClusterConfiguration(vm_size="STANDARD_D2_V2")
    cpu_cluster = ml_client.compute.create(cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True)

    

Found existing cluster, use it.


In [16]:

# get the data asset
data_asset = ml_client.data.get(name="multistacc_mltable_validation", version="2")
# enviromment =  ml_client.environments.get(name="mltable_env", version="11")
enviromment = Environment(
    build=BuildContext(path="env"),
    name="mltable_env_vinimage",
    description="Environment created from a Docker context.",
)

job = command(
    command="python read-mltable.py --input ${{inputs.my_mltable}}",
    inputs={
        "my_mltable": Input(type="mltable",path=data_asset.id)
    },
    compute="cpu-cluster",
    # environment=enviromment,
    environment=enviromment,
    code="./src"
)

ml_client.jobs.create_or_update(job)

Uploading env (0.0 MBs): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1419/1419 [00:00<00:00, 42222.14it/s]


Experiment,Name,Type,Status,Details Page
basic-job,frosty_candle_bcpsp0wp7x,command,Starting,Link to Azure Machine Learning studio
